Collect  tweets  published  via  Endomondo  by  people  from  3-4  major  cities  in Poland.   Check  if  there  are  some  differences  in  types  of  activities  between them.  Visualize the activity areas on a map.

In [3]:
import json
import tweepy

In [21]:
# Getting the authentication keys and tokens
with open('twitter_config.json', mode='r') as file:
    config = json.load(file)

In [24]:
# Authenticating to Twitter
auth = tweepy.OAuthHandler(config['consumerKey'], config['consumerSecret'])
auth.set_access_token(config['accessToken'], config['accessTokenSecret'])

In [25]:
# Creating an API object
api = tweepy.API(auth)

In [26]:
# Creating a tweet!
api.update_status(status='Hello Twitter from python program', 
                  lat=51.107885, 
                  long=17.038538)

Status(_api=<tweepy.api.API object at 0x000001A3A8BB3E10>, _json={'created_at': 'Wed Nov 20 22:31:21 +0000 2019', 'id': 1197281302317293569, 'id_str': '1197281302317293569', 'text': 'Hello Twitter from python program', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []}, 'source': '<a href="https://www.twitter.com" rel="nofollow">poland-cities-endomondo-analysis</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1197223796102578177, 'id_str': '1197223796102578177', 'name': 'Kajetan', 'screen_name': 'Kajetan24191735', 'location': '', 'description': '', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 0, 'friends_count': 0, 'listed_count': 0, 'created_at': 'Wed Nov 20 18:43:29 +0000 2019', 'favourites_count': 0, 'utc_offset': None, 'time_zone': None, 'geo_enabled': Fals